## Required Imports

#### If necessary: Sampling lib - https://little-ball-of-fur.readthedocs.io/en/latest/notes/installation.html

In [ ]:
import networkx as nx
from networkx.algorithms import community
import pickle as pkl
from sklearn.preprocessing import LabelEncoder
import random
import numpy as np
import pandas as pd
import logging
from littleballoffur import RandomNodeSampler, ForestFireSampler
from littleballoffur import RandomWalkSampler
import sys
import networkx as nx
import igraph as ig
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_mutual_info_score
import csv
import random
import pickle as pkl
# import community
# from community import community_louvain
from collections import Counter
import pandas as pd
import numpy as np
import sys
import igraph as ig
import pickle
from sklearn.metrics.cluster import normalized_mutual_info_score
import pandas as pd
from igraph.clustering import compare_communities
import sys, warnings

In [ ]:
edgelist_path = 'whatsapp.edgelist'  #Insert here dataset edgelist file path (in this code we have whatsapp example)
G = nx.read_weighted_edgelist(edgelist_path, delimiter=" ") #Read 


In [ ]:
graph = G
number_of_nodes = graph.number_of_nodes()
print(number_of_nodes)
print(graph.number_of_edges())
# validate dataset number of edges and nodes

# Original Network

## 1. Execute 10x community detection in original network

##### I store in a csv to have a backup for further analysis

In [ ]:
def characterize_network_executions(G, Path_Communities, header,index):
    list_results = []
                
    n_nodes = len(G.nodes())
    n_edges = len(G.edges())
    
    degree = dict(G.degree)
    avg_degree = round(np.average(list(degree.values())),4)
    std_degree = round(np.std(list(degree.values())),4)
    
    edge_weights = nx.get_edge_attributes(G, 'weight')
    edge_weights_values = list(edge_weights.values())
    avg_weight = np.mean(edge_weights_values)
    std_weight = np.std(edge_weights_values)
    
    density = round(nx.density(G),4)
    
    n_cc = nx.number_connected_components(G)
    components = list(nx.connected_components(G))
    component_sizes = [len(component) for component in components]
    largest_component_size = max(component_sizes)
    
    clustering = round(nx.average_clustering(G),4)
    clustering_coeffs = nx.clustering(G)
    clustering_coeffs_values = list(clustering_coeffs.values())
    std_clustering = np.std(clustering_coeffs_values)
    
    # Calculate Louvain community detection
    partition = community.best_partition(G, resolution=1)
    modularity = community.modularity(partition, G)
    
    partition = {node: comm_id for node, comm_id in partition.items() if list(partition.values()).count(comm_id) > 10}
    
    pkl.dump(partition, open(Path_Communities, "wb"), protocol=4)
    
    n_comm = len(set(partition.values()))
    community_sizes = Counter(partition.values())
    biggest_community_size = max(community_sizes.values())
    
    
    print(n_nodes, n_edges, n_cc, modularity)
    
    assortativity = nx.degree_assortativity_coefficient(G)
    
    parameter = '-'
    sample = "-"
    batch = "-"
    intersection = "-"
    nmi = "-"
    
    network = 'whatsapp' # network name
    type_network = "original_"+str(i)
    snap = "October"
    
    list_results.append((network, type_network, snap, n_nodes, n_edges, avg_weight, std_weight, avg_degree, std_degree, 
                         density, clustering, std_clustering, n_cc, largest_component_size, n_comm,biggest_community_size, modularity, assortativity, parameter, sample, batch,intersection, nmi))
    
    df = pd.DataFrame(list_results, columns=['Network', 'Type', 'Snapshot', '# Nodes', '# Edges', 'Avg. Weight', 'Std. Weight',
                                             'Avg. Degree','Std. Degree','Density', 'Avg. Clustering', 'Std. Clustering','# Components',
                                             '# Big. Component Size','# Communities', '# Big. Community Size', 'Modularity', "Assortivity", 
                                             "Parameter","Net. Sample", "Batch", "intersection","nmi"])

    df.to_csv("network/original/network_whatsapp_original_characterization.csv", mode='a', header=header, index=None)

In [ ]:
header = True
for i in range(1, 11):
    characterize_network_executions(graph, 'whatsapp/original/comm_complete_network_'+str(i)+'.pkl', header, i)
    header = False

### 1.1 Compare communities of the executions and calculate NMI mean and std

In [ ]:
min_tamanho_comunidade = 10
PATH_Communities = 'whatsapp/original/'
list_results = []
list_models = ['comm_complete_whatsapp_1', 'comm_complete_whatsapp_2', 
               'comm_complete_whatsapp_3', 'comm_complete_whatsapp_4', 
               'comm_complete_whatsapp_5', 'comm_complete_whatsapp_6', 
               'comm_complete_whatsapp_7', 'comm_complete_whatsapp_8',
               'comm_complete_whatsapp_9', 'comm_complete_whatsapp_10']

for i in range(1,len(list_models),1):
    model_1 = list_models[0]
    model_2 = list_models[i]
  
    # Load the pickled communities
    with open(PATH_Communities + model_1 + '.pkl', 'rb') as file:
        dicionario1 = pickle.load(file)
    with open(PATH_Communities + model_2 + '.pkl', 'rb') as file:
        dicionario2 = pickle.load(file)

    # Contagem do tamanho das comunidades e filtragem
    comunidades_filtradas1 = {}
    comunidades_filtradas2 = {}

    # Filtra as comunidades com tamanho maior ou igual a min_tamanho_comunidade
    for node, community_id in dicionario1.items():
        tamanho_comunidade = list(dicionario1.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas1[node] = community_id

    for node, community_id in dicionario2.items():
        tamanho_comunidade = list(dicionario2.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas2[node] = community_id

    # Obter as chaves em comum entre os dicionários filtrados
    chaves_comuns = set(comunidades_filtradas1.keys()) & set(comunidades_filtradas2.keys())
    intersection = len(chaves_comuns)
    # print(intersection)
    # Inicializar as listas para armazenar os valores correspondentes
    valores_dicionario1 = []
    valores_dicionario2 = []

    # Preencher as listas com os valores correspondentes, mantendo a ordem
    for chave in chaves_comuns:
        valores_dicionario1.append(comunidades_filtradas1[chave])
        valores_dicionario2.append(comunidades_filtradas2[chave])

    nmi = 0
    # Calcular o NMI usando scikit-learn
    if(intersection> 1):
        nmi = normalized_mutual_info_score(valores_dicionario1, valores_dicionario2)

    list_results.append((model_1, model_2, nmi))

df = pd.DataFrame(list_results, columns = ['Model 1', 'Model 2', 'NMI'])

# Save the DataFrame as a CSV file
csv_file = 'whatsapp/original/nmi_original_whatsapp_results.csv'
df.to_csv(csv_file, index=False)

df

mean_nmi = df["NMI"].mean()
std_nmi = df["NMI"].std()

# Create a new DataFrame with the mean and std values
result_df = pd.DataFrame({"NMI": ['{:.4f}±{:.4f}'.format(mean_nmi,std_nmi)]})

print(result_df)

csv_file = 'whatsapp/original/nmi_mean_whatsapp_results.csv'
result_df.to_csv(csv_file, index=False)

## 2 - Execute Random Node Method to generate original network samples (we have generated 10samples for each param)

#### Created relabel method to handle limitation of networkX
#### Random Node Reference: https://little-ball-of-fur.readthedocs.io/en/latest/_modules/littleballoffur/node_sampling/randomnodesampler.html#RandomNodeSampler
#### I also stored samples to be able to detect its communities next.

In [ ]:
def relabel_nodes(graph):
    nodes = sorted(graph.nodes)
    relabel_mapping = {node: idx for idx, node in enumerate(nodes)}
    relabeled_graph = nx.relabel_nodes(graph, relabel_mapping)
    
    # Update node weights in the relabeled graph
    for u, v, data in graph.edges(data=True):
        if u in relabel_mapping and v in relabel_mapping:
            u_mapped = relabel_mapping[u]
            v_mapped = relabel_mapping[v]
            if u_mapped in relabeled_graph and v_mapped in relabeled_graph[u_mapped]:
                relabeled_graph[u_mapped][v_mapped]['weight'] = data['weight']
    
    return relabeled_graph, relabel_mapping

def revert_relabel_nodes(graph, relabel_mapping):
    original_mapping = {v: k for k, v in relabel_mapping.items()}
    reverted_graph = nx.relabel_nodes(graph, original_mapping)
    return reverted_graph

def run_random_node_sampling_model(graph_nx, dataset):
    j = 1
    df = pd.DataFrame()
    
    # Step 1: Apply relabel_nodes(graph) to weighted graph_nx
    relabeled_graph_nx, relabel_mapping = relabel_nodes(graph_nx)
    
    while j <= 10:
        print(j)
        values_seed = []
        seed = random.randint(0, 4294967294)
        
        for i in range(1, 11, 1):
            random_node_model = RandomNodeSampler(number_of_nodes=int((i/10)*relabeled_graph_nx.number_of_nodes()), seed=seed)
            
            # Step 2: Use the weighted relabeled_graph to calculate sample
            sample_random_node = random_node_model.sample(relabeled_graph_nx)
            
            # Step 3: Revert relabeling to original nodes mapped in relabel_mapping
            relabeled_sample_random_node = revert_relabel_nodes(sample_random_node, relabel_mapping)
            
            result = graph_nx.subgraph(relabeled_sample_random_node.nodes()).edges() == relabeled_sample_random_node.edges()
            print(result)
                
            df_random_node = nx.to_pandas_edgelist(relabeled_sample_random_node, source='source', target='target', nodelist=list(relabeled_sample_random_node.nodes), dtype=None, edge_key=None)

            df_random_node.to_csv("whatsapp/RandomNode/samples/"+dataset+"_"+str(i/10)+'_random_node_edge_list_batch_'+str(j)+'_.csv',index=False)
            print("saved_"+dataset+"_"+str(i/10)+'_random_node_edge_list_batch_'+str(j)+'_'+str(seed)+'_.csv')
            
            values_seed.append(seed)
        
        df['batch_seed_'+str(j)] = values_seed
        df.to_csv("whatsapp/RandomNode/samples/whatsapp_seeds.csv",index=False)
        del seed
        j += 1
    
    return df

In [ ]:
run_random_node_sampling_model(G, 'whatsapp')

## 3. Detect community of original network generated samples - Random Node

##### In this scenario I have created a separeted method called characterize_random_node_sample_networks to call characterize_sampled_network to detect the  communities of each sample and, after that, I call in it a separated method calculate_nmi to calcutate NMI of the samples.
##### I also store the communities info in a separated files for further analysis and I put all the samples info in another csv.

In [ ]:
def calculate_nmi(dicionario1_pkl, dicionario2_pkl, min_tamanho_comunidade=10):
    #from sklearn.metrics.cluster import adjusted_mutual_info_score
    # Carregar os dicionários a partir dos arquivos pickle
    with open(dicionario1_pkl, 'rb') as arquivo1:
        dict1 = pkl.load(arquivo1)
    
    with open(dicionario2_pkl, 'rb') as arquivo2:
        dict2 = pkl.load(arquivo2)
        
    dicionario1 = {int(key): value for key, value in dict1.items()}
    dicionario2 = {int(key): value for key, value in dict2.items()}

    # Contagem do tamanho das comunidades e filtragem
    comunidades_filtradas1 = {}
    comunidades_filtradas2 = {}
    
    # Filtra as comunidades com tamanho maior ou igual a min_tamanho_comunidade
    for node, community_id in dicionario1.items():
        tamanho_comunidade = list(dicionario1.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas1[node] = community_id

    for node, community_id in dicionario2.items():
        tamanho_comunidade = list(dicionario2.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas2[node] = community_id
    
    chaves_comuns = list(set(comunidades_filtradas1.keys()).intersection(comunidades_filtradas2.keys()))
    
    intersection = len(chaves_comuns)
    print(intersection)
    # Inicializar as listas para armazenar os valores correspondentes
    valores_dicionario1 = []
    valores_dicionario2 = []
    
    # Preencher as listas com os valores correspondentes, mantendo a ordem
    for chave in chaves_comuns:
        valores_dicionario1.append(comunidades_filtradas1[chave])
        valores_dicionario2.append(comunidades_filtradas2[chave])
    
    if(intersection <= 1): return 0, 0

    # Calcular o NMI usando scikit-learn
    nmi = normalized_mutual_info_score(valores_dicionario1, valores_dicionario2)
    print(nmi)
    
    return nmi, intersection

In [ ]:
def characterize_sampled_network(graph, Path_Communities, network, type_network, snap, s, parameter, header, sample, batch):
    
    #Para rede amostrada
    list_results = []
    print(graph)
    df = pd.read_csv(graph)
    Graphtype = nx.Graph()
    G_sampled = nx.from_pandas_edgelist(df, edge_attr='weight', create_using=Graphtype)
    
    print(df[:3])
    
    for component in list(nx.connected_components(G_sampled)):
        if len(component)<3:
            for node in component:
                G_sampled.remove_node(node)
                
    n_nodes = len(G_sampled.nodes())
    n_edges = len(G_sampled.edges())
    
    degree = dict(G_sampled.degree)
    avg_degree = round(np.average(list(degree.values())),4)
    std_degree = round(np.std(list(degree.values())),4)
    
    edge_weights = nx.get_edge_attributes(G_sampled, 'weight')
    edge_weights_values = list(edge_weights.values())
    avg_weight = np.mean(edge_weights_values)
    std_weight = np.std(edge_weights_values)
    
    density = round(nx.density(G_sampled),4)
    
    n_cc = nx.number_connected_components(G_sampled)
    components = list(nx.connected_components(G_sampled))
    component_sizes = [len(component) for component in components]
    largest_component_size = max(component_sizes)
    
    clustering = round(nx.average_clustering(G_sampled),4)
    clustering_coeffs = nx.clustering(G_sampled)
    clustering_coeffs_values = list(clustering_coeffs.values())
    std_clustering = np.std(clustering_coeffs_values)
    
    
    complete_partition_sampled = community.best_partition(G_sampled, resolution=1)
    modularity = community.modularity(complete_partition_sampled, G_sampled)
    
    pkl.dump(complete_partition_sampled, open(Path_Communities, "wb"), protocol=4) 
    
    n_comm_10 = len(set(complete_partition_sampled.values()))
    
    community_sizes_10 = Counter(complete_partition_sampled.values())
    
    biggest_community_size_10 = max(community_sizes_10.values())
    
    print(n_nodes, n_edges, n_cc, modularity)
    
    assortativity = nx.degree_assortativity_coefficient(G_sampled)
    
    # Calcula o NMI entre os arquivos pickle
    nmi, intersection = calculate_nmi('whatsapp/original/comm_complete_whatsapp_1.pkl', Path_Communities)
    
    parameter = parameter
    sample = sample
    batch = batch
    
    network = 'WhatsApp'
    type_network = "sample_RN"
    snap = "October"
    
    list_results.append((network, type_network, snap, n_nodes, n_edges, avg_weight, std_weight, avg_degree, std_degree, 
                         density, clustering, std_clustering, n_cc, largest_component_size, n_comm_10,biggest_community_size_10, modularity, assortativity, parameter, sample, batch,intersection, nmi))
    
    df = pd.DataFrame(list_results, columns=['Network', 'Type', 'Snapshot', '# Nodes', '# Edges', 'Avg. Weight', 'Std. Weight',
                                             'Avg. Degree','Std. Degree','Density', 'Avg. Clustering', 'Std. Clustering','# Components',
                                             '# Big. Component Size','# Communities', '# Big. Community Size', 'Modularity', "Assortivity", 
                                             "Parameter","Net. Sample", "Batch", "intersection","nmi"])
    del G_sampled
    # del G
    # del partition
    del complete_partition_sampled

    df.to_csv("whatsapp/RandomNode/communities/networks_whatsapp_samples_characterization.csv", mode='a', header=header, index=None)

In [ ]:
def characterize_random_node_sample_networks(graph, dataset):
    header = True
    j = 1
    
    while j <= 10:
        print(j)
        for i in range(1,11,1):
            sample = "whatsapp/RandomNode/samples/"+dataset+"_"+str(i/10)+'_random_node_edge_list_batch_'+str(j)+'_.csv'
            Path_Communities = "whatsapp/RandomNode/communities/"+dataset+"_RN_batch_"+str(j)+"_net_percent_"+str(i/10)+".pkl"
            characterize_sampled_network(sample, Path_Communities, dataset, "sample whatsapp - Random Node", "sample_"+str(i)+"_batch_"+str(j), 1, 0, header, str(i/10), str(j))
            # print("saved_community_"+dataset+"_"+str(i/10)+'_random_node_edge_list_batch_'+str(j)+'_.csv')
            header = False
        header = False
        j += 1

### 3.1 Calculate samples NMI mean and std per parameter

#### In this code I load csv with samples and samples communities info and I group them per parameter and after that I calculate NMI mean and std
#### At the end I store them in a csv for further analysis

In [ ]:
df_random_node_original_whatsapp = pd.read_csv('RandomNode/communities/networks_whatsapp_samples_characterization.csv')


In [ ]:
df_RN_NMI_whatsapp = df_random_node_original_whatsapp

In [ ]:
df_RN_NMI_whatsapp[:3]

In [ ]:
df_RN_NMI_whatsapp["nmi_mean"]=0
df_RN_NMI_whatsapp["std"]=0
df_RN_NMI_whatsapp["nmi_std"] = 0

In [ ]:
df_RN_NMI_whatsapp['nmi_norm'] = df_RN_NMI_whatsapp['nmi'] / 0.4552

In [ ]:
df_RN_NMI_whatsapp['nmi_mean'] = df_RN_NMI_whatsapp.groupby(['Net. Sample'])['nmi_norm'].transform(lambda x: x.mean())

In [ ]:
df_RN_NMI_whatsapp['std'] = df_RN_NMI_whatsapp.groupby(['Net. Sample'])['nmi_norm'].transform(lambda x: x.std(ddof=1))

In [ ]:
df_RN_NMI_whatsapp["nmi_std"] = df_RN_NMI_whatsapp.apply(lambda x: '{:.4f}±{:.4f}'.format(x["nmi_mean"], x["std"]), axis=1)

In [ ]:
new_df_2 = df_RN_NMI_whatsapp[['Batch', 'Net. Sample','nmi_std']].copy()
new_df_2[:3]

In [ ]:
pivoted_2 = new_df_2.pivot(index=["Batch"], columns="Net. Sample")
pd.set_option('display.max_rows', None)

In [ ]:
pivoted_2[:1].to_csv("RandomNode/communities/networks_nmi_whatsapp_samples_normalized.csv")

## 4 - Execute Forest Fire Method to generate original network samples (we have generated 10 samples for each param)

#### Created relabel method to handle limitation of networkX
#### Forest Fire Reference: https://little-ball-of-fur.readthedocs.io/en/latest/_modules/littleballoffur/exploration_sampling/forestfiresampler.html#ForestFireSampler
#### I also stored samples to be able to detect its communities next.

In [ ]:
def relabel_nodes(graph):
    nodes = sorted(graph.nodes)
    relabel_mapping = {node: idx for idx, node in enumerate(nodes)}
    relabeled_graph = nx.relabel_nodes(graph, relabel_mapping)
    
    # Update node weights in the relabeled graph
    for u, v, data in graph.edges(data=True):
        if u in relabel_mapping and v in relabel_mapping:
            u_mapped = relabel_mapping[u]
            v_mapped = relabel_mapping[v]
            if u_mapped in relabeled_graph and v_mapped in relabeled_graph[u_mapped]:
                relabeled_graph[u_mapped][v_mapped]['weight'] = data['weight']
    
    return relabeled_graph, relabel_mapping

def revert_relabel_nodes(graph, relabel_mapping):
    original_mapping = {v: k for k, v in relabel_mapping.items()}
    reverted_graph = nx.relabel_nodes(graph, original_mapping)
    return reverted_graph

def run_forest_fire_sampling_model(graph_nx,dataset):
    k = 1
    df = pd.DataFrame()
    
    relabeled_graph_nx, relabel_mapping = relabel_nodes(graph_nx)
    
    while k <= 10:
        print(k)
        for i in range(1,11,1):
            values_seed = []
            for j in range(1,10,1):
                seed = random.randint(0, 4294967294)
                forest_fire_model = ForestFireSampler(number_of_nodes=int((i/10)*relabeled_graph_nx.number_of_nodes()),p=(j/10),seed=seed)
                
                sample_forest_fire = forest_fire_model.sample(relabeled_graph_nx)
                
                relabeled_sample_forest_fire = revert_relabel_nodes(sample_forest_fire, relabel_mapping)
            
                result = graph_nx.subgraph(relabeled_sample_forest_fire.nodes()).edges() == relabeled_sample_forest_fire.edges()
                print(result)
                
                df_forest_fire_node = nx.to_pandas_edgelist(relabeled_sample_forest_fire, source='source', target='target', nodelist=list(relabeled_sample_forest_fire.nodes), dtype=None, edge_key=None)
                df_forest_fire_node.to_csv("whatsapp/ForestFire/samples/"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch'+str(k)+'_'+str(i)+'.csv',index=False)
                print("whatsapp/ForestFire/samples/"+"dataset_"+dataset+"/"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch'+str(k)+'_'+str(i)+'.csv')
                del forest_fire_model
                del sample_forest_fire
                del df_forest_fire_node
                values_seed.append(seed)
            df['batch_seed_'+str(k)] = values_seed
            df.to_csv("whatsapp/ForestFire/samples/"+dataset+"_forest_fire_seeds.csv",index=False)
            print(values_seed)
        k += 1

In [ ]:
run_forest_fire_sampling_model(G, 'whatsapp')

## 5. Detect community of original network generated samples - Forest Fire

##### In this scenario I have created a separeted method called characterize_forest_fire_sample_networks to call characterize_sampled_network_ff to detect the  communities of each sample and, after that, I call in it a separated method calculate_nmi to calcutate NMI of the samples.
##### I also store the communities info in a separated files for further analysis and I put all the samples info in another csv.

In [ ]:
def calculate_nmi(dicionario1_pkl, dicionario2_pkl, min_tamanho_comunidade=10):
    #from sklearn.metrics.cluster import adjusted_mutual_info_score
    # Carregar os dicionários a partir dos arquivos pickle
    with open(dicionario1_pkl, 'rb') as arquivo1:
        dict1 = pkl.load(arquivo1)
    
    with open(dicionario2_pkl, 'rb') as arquivo2:
        dict2 = pkl.load(arquivo2)
        
    dicionario1 = {int(key): value for key, value in dict1.items()}
    dicionario2 = {int(key): value for key, value in dict2.items()}

    # Contagem do tamanho das comunidades e filtragem
    comunidades_filtradas1 = {}
    comunidades_filtradas2 = {}
    
    # Filtra as comunidades com tamanho maior ou igual a min_tamanho_comunidade
    for node, community_id in dicionario1.items():
        tamanho_comunidade = list(dicionario1.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas1[node] = community_id

    for node, community_id in dicionario2.items():
        tamanho_comunidade = list(dicionario2.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas2[node] = community_id
    
    chaves_comuns = list(set(comunidades_filtradas1.keys()).intersection(comunidades_filtradas2.keys()))
    
    intersection = len(chaves_comuns)
    print(intersection)
    # Inicializar as listas para armazenar os valores correspondentes
    valores_dicionario1 = []
    valores_dicionario2 = []
    
    # Preencher as listas com os valores correspondentes, mantendo a ordem
    for chave in chaves_comuns:
        valores_dicionario1.append(comunidades_filtradas1[chave])
        valores_dicionario2.append(comunidades_filtradas2[chave])
    
    if(intersection < 1): return 0, 0

    # Calcular o NMI usando scikit-learn
    nmi = normalized_mutual_info_score(valores_dicionario1, valores_dicionario2)
    print(nmi)
    
    return nmi, intersection

In [ ]:
def characterize_sampled_network_ff(graph, Path_Communities, network, type_network, snap, s, parameter, header, sample, batch):
    
    #Para rede amostrada
    list_results = []
    print(graph)
    df = pd.read_csv(graph)
    Graphtype = nx.Graph()
    G_sampled = nx.from_pandas_edgelist(df, edge_attr='weight', create_using=Graphtype)
    
    print(df[:3])
    
    # for component in list(nx.connected_components(G_sampled)):
    #     if len(component)<3:
    #         for node in component:
    #             G_sampled.remove_node(node)
                
    n_nodes = len(G_sampled.nodes())
    n_edges = len(G_sampled.edges())
    
    degree = dict(G_sampled.degree)
    avg_degree = round(np.average(list(degree.values())),4)
    std_degree = round(np.std(list(degree.values())),4)
    
    edge_weights = nx.get_edge_attributes(G_sampled, 'weight')
    edge_weights_values = list(edge_weights.values())
    avg_weight = np.mean(edge_weights_values)
    std_weight = np.std(edge_weights_values)
    
    density = round(nx.density(G_sampled),4)
    
    n_cc = nx.number_connected_components(G_sampled)
    components = list(nx.connected_components(G_sampled))
    component_sizes = [len(component) for component in components]
    largest_component_size = max(component_sizes)
    
    clustering = round(nx.average_clustering(G_sampled),4)
    clustering_coeffs = nx.clustering(G_sampled)
    clustering_coeffs_values = list(clustering_coeffs.values())
    std_clustering = np.std(clustering_coeffs_values)
    
    
    complete_partition_sampled = community.best_partition(G_sampled, resolution=1)
    modularity = community.modularity(complete_partition_sampled, G_sampled)
    
    pkl.dump(complete_partition_sampled, open(Path_Communities, "wb"), protocol=4) 
    
    n_comm_10 = len(set(complete_partition_sampled.values()))
    
    community_sizes_10 = Counter(complete_partition_sampled.values())
    
    biggest_community_size_10 = max(community_sizes_10.values())
    
    print(n_nodes, n_edges, n_cc, modularity)
    
    assortativity = nx.degree_assortativity_coefficient(G_sampled)
    
    # Calcula o NMI entre os arquivos pickle
    nmi, intersection = calculate_nmi('whatsapp/original/comm_complete_whatsapp_1.pkl', Path_Communities)
    
    parameter = parameter
    sample = sample
    batch = batch
    
    network = 'WhatsApp'
    type_network = "sample_FF"
    snap = "October"
    
    list_results.append((network, type_network, snap, n_nodes, n_edges, avg_weight, std_weight, avg_degree, std_degree, 
                         density, clustering, std_clustering, n_cc, largest_component_size, n_comm_10,biggest_community_size_10, modularity, assortativity, parameter, sample, batch,intersection, nmi))
    
    df = pd.DataFrame(list_results, columns=['Network', 'Type', 'Snapshot', '# Nodes', '# Edges', 'Avg. Weight', 'Std. Weight',
                                             'Avg. Degree','Std. Degree','Density', 'Avg. Clustering', 'Std. Clustering','# Components',
                                             '# Big. Component Size','# Communities', '# Big. Community Size', 'Modularity', "Assortivity", 
                                             "Parameter","Net. Sample", "Batch", "intersection","nmi"])
    del G_sampled
    # del G
    # del partition
    del complete_partition_sampled

    df.to_csv("whatsapp/ForestFire/communities/networks_whatsapp_samples_characterization.csv", mode='a', header=header, index=None)

In [ ]:
def characterize_forest_fire_sample_networks(graph, dataset):
    header = True
    k = 1
    while k <= 10:
        print(k)
        for i in range(1,11,1):
            for j in range(1,10,1):
                sample = "whatsapp/ForestFire/samples/"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch'+str(k)+'_'+str(i)+'.csv'
                Path_Communities = "whatsapp/ForestFire/communities/"+dataset+"_FF_batch_"+str(k)+"_net_percent_"+str(i/10)+"_probability_"+str(j/10)+".pkl"
                characterize_sampled_network_ff(sample, Path_Communities, dataset, "sample Whatsapp - Forest Fire", "batch_"+str(k)+'_sample_'+str(i/10)+'_probaability_'+str(j/10), 1, str(j/10), header, str(i/10), str(k))
                header = False
            # print("saved_community_"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch_'+str(k)+'_'+str(i)+'_.csv')
            header = False
        header = False
        k += 1

In [ ]:
# characterize_forest_fire_sample_networks(graph, 'whatsapp')

### 5.1 Calculate samples NMI mean and std per parameter

#### In this code I load csv with samples and samples communities info and I group them per parameter and after that I calculate NMI mean and std
#### At the end I store them in a csv for further analysis

In [ ]:
df_ff_original_wpp = pd.read_csv('ForestFire/communities/networks_whatsapp_samples_characterization.csv')


In [ ]:
df_FF_NMI_wpp = df_ff_original_wpp

In [ ]:
df_ff_original_wpp[:3]

In [ ]:
df_FF_NMI_wpp["nmi_mean"]=0
df_FF_NMI_wpp["std_1"]=0
df_FF_NMI_wpp["nmi_std"] = 0

In [ ]:
df_FF_NMI_wpp['nmi_norm'] = df_FF_NMI_wpp['nmi'] / 0.4552

In [ ]:
df_FF_NMI_wpp['nmi_mean'] = df_FF_NMI_wpp.groupby(['Net. Sample','Parameter'])['nmi_norm'].transform(lambda x: x.mean())

In [ ]:
df_FF_NMI_wpp['std_1'] = df_FF_NMI_wpp.groupby(['Net. Sample','Parameter'])['nmi_norm'].transform(lambda x: x.std(ddof=1))

In [ ]:
df_FF_NMI_wpp["nmi_std"] = df_FF_NMI_wpp.apply(lambda x: '{:.4f}±{:.4f}'.format(x["nmi_mean"], x["std_1"]), axis=1)

In [ ]:
new_df_2 = df_FF_NMI_wpp[['Batch','Net. Sample', 'Parameter','nmi_std']].copy()
new_df_2[:3]

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# new_df_2[:100]

In [ ]:
rslt_df_2 = new_df_2[new_df_2['Batch'] == 1]
rslt_df_2.drop('Batch', axis=1, inplace=True)

In [ ]:
pivoted_2 = rslt_df_2.pivot(index=["Net. Sample"], columns=["Parameter"])
pd.set_option('display.max_rows', None)
pivoted_2[:100] 

In [ ]:
pivoted_2[:100].to_csv("ForestFire/communities/networks_whatsapp_samples_nmi_normalized.csv", mode='a', header=True, index=None)

## 6 - Execute Random Walk Method to generate original network samples (we have generated 10 samples for each param)

#### Created relabel method to handle limitation of networkX
#### Random Walk Reference: https://little-ball-of-fur.readthedocs.io/en/latest/_modules/littleballoffur/exploration_sampling/randomwalksampler.html#RandomWalkSampler
#### I also stored samples to be able to detect its communities next.

In [ ]:
def relabel_nodes(graph):
    nodes = sorted(graph.nodes)
    relabel_mapping = {node: idx for idx, node in enumerate(nodes)}
    relabeled_graph = nx.relabel_nodes(graph, relabel_mapping)
    
    # Update node weights in the relabeled graph
    for u, v, data in graph.edges(data=True):
        if u in relabel_mapping and v in relabel_mapping:
            u_mapped = relabel_mapping[u]
            v_mapped = relabel_mapping[v]
            if u_mapped in relabeled_graph and v_mapped in relabeled_graph[u_mapped]:
                relabeled_graph[u_mapped][v_mapped]['weight'] = data['weight']
    
    return relabeled_graph, relabel_mapping

def revert_relabel_nodes(graph, relabel_mapping):
    original_mapping = {v: k for k, v in relabel_mapping.items()}
    reverted_graph = nx.relabel_nodes(graph, original_mapping)
    return reverted_graph

def run_random_walk_sampling_model(graph_nx, dataset):
    j = 1
    df = pd.DataFrame()
    
    # Step 1: Apply relabel_nodes(graph) to weighted graph_nx
    relabeled_graph_nx, relabel_mapping = relabel_nodes(graph_nx)
    
    while j <= 10:
        print(j)
        values_seed = []
        seed = random.randint(0, 4294967294)
        
        for i in range(1, 10, 1):
            random_node_model = RandomWalkSampler(number_of_nodes=int((i/10)*relabeled_graph_nx.number_of_nodes()), seed=seed)
            
            # Step 2: Use the weighted relabeled_graph to calculate sample
            sample_random_node = random_node_model.sample(relabeled_graph_nx)
            
            # Step 3: Revert relabeling to original nodes mapped in relabel_mapping
            relabeled_sample_random_node = revert_relabel_nodes(sample_random_node, relabel_mapping)
            
            result = graph_nx.subgraph(relabeled_sample_random_node.nodes()).edges() == relabeled_sample_random_node.edges()
            print(result)
                
            df_random_node = nx.to_pandas_edgelist(relabeled_sample_random_node, source='source', target='target', nodelist=list(relabeled_sample_random_node.nodes), dtype=None, edge_key=None)

            df_random_node.to_csv("whatsapp/RandomWalk/samples/"+dataset+"_"+str(i/10)+'_random_walk_edge_list_batch_'+str(j)+'_.csv',index=False)
            print("saved_"+dataset+"_"+str(i/10)+'_random_walk_edge_list_batch_'+str(j)+'_'+str(seed)+'_.csv')
            
            values_seed.append(seed)
        
        df['batch_seed_'+str(j)] = values_seed
        df.to_csv("whatsapp/RandomWalk/samples/whatsapp_seeds.csv",index=False)
        del seed
        j += 1
    
    return df

In [ ]:
# run_random_walk_sampling_model(graph, 'whatsapp')

## 7. Detect community of original network generated samples - Random Walk

##### In this scenario I have created a separeted method called characterize_random_walk_sample_networks to call characterize_sampled_network to detect the  communities of each sample and, after that, I call in it a separated method calculate_nmi to calcutate NMI of the samples.
##### I also store the communities info in a separated files for further analysis and I put all the samples info in another csv.

In [ ]:
def calculate_nmi(dicionario1_pkl, dicionario2_pkl, min_tamanho_comunidade=10):
    #from sklearn.metrics.cluster import adjusted_mutual_info_score
    # Carregar os dicionários a partir dos arquivos pickle
    with open(dicionario1_pkl, 'rb') as arquivo1:
        dict1 = pkl.load(arquivo1)
    
    with open(dicionario2_pkl, 'rb') as arquivo2:
        dict2 = pkl.load(arquivo2)
        
    dicionario1 = {int(key): value for key, value in dict1.items()}
    dicionario2 = {int(key): value for key, value in dict2.items()}

    # Contagem do tamanho das comunidades e filtragem
    comunidades_filtradas1 = {}
    comunidades_filtradas2 = {}
    
    # Filtra as comunidades com tamanho maior ou igual a min_tamanho_comunidade
    for node, community_id in dicionario1.items():
        tamanho_comunidade = list(dicionario1.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas1[node] = community_id

    for node, community_id in dicionario2.items():
        tamanho_comunidade = list(dicionario2.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas2[node] = community_id
    
    chaves_comuns = list(set(comunidades_filtradas1.keys()).intersection(comunidades_filtradas2.keys()))
    
    intersection = len(chaves_comuns)
    print(intersection)
    # Inicializar as listas para armazenar os valores correspondentes
    valores_dicionario1 = []
    valores_dicionario2 = []
    
    # Preencher as listas com os valores correspondentes, mantendo a ordem
    for chave in chaves_comuns:
        valores_dicionario1.append(comunidades_filtradas1[chave])
        valores_dicionario2.append(comunidades_filtradas2[chave])
    
    if(intersection <= 1): return 0, 0

    # Calcular o NMI usando scikit-learn
    nmi = normalized_mutual_info_score(valores_dicionario1, valores_dicionario2)
    print(nmi)
    
    return nmi, intersection

In [ ]:
def characterize_sampled_network(graph, Path_Communities, network, type_network, snap, s, parameter, header, sample, batch):
    
    #Para rede amostrada
    list_results = []
    print(graph)
    df = pd.read_csv(graph)
    Graphtype = nx.Graph()
    G_sampled = nx.from_pandas_edgelist(df, edge_attr='weight', create_using=Graphtype)
    
    print(df[:3])
    
    # for component in list(nx.connected_components(G_sampled)):
    #     if len(component)<3:
    #         for node in component:
    #             G_sampled.remove_node(node)
                
    n_nodes = len(G_sampled.nodes())
    n_edges = len(G_sampled.edges())
    
    degree = dict(G_sampled.degree)
    avg_degree = round(np.average(list(degree.values())),4)
    std_degree = round(np.std(list(degree.values())),4)
    
    edge_weights = nx.get_edge_attributes(G_sampled, 'weight')
    edge_weights_values = list(edge_weights.values())
    avg_weight = np.mean(edge_weights_values)
    std_weight = np.std(edge_weights_values)
    
    density = round(nx.density(G_sampled),4)
    
    n_cc = nx.number_connected_components(G_sampled)
    components = list(nx.connected_components(G_sampled))
    component_sizes = [len(component) for component in components]
    largest_component_size = max(component_sizes)
    
    clustering = round(nx.average_clustering(G_sampled),4)
    clustering_coeffs = nx.clustering(G_sampled)
    clustering_coeffs_values = list(clustering_coeffs.values())
    std_clustering = np.std(clustering_coeffs_values)
    
    
    complete_partition_sampled = community.best_partition(G_sampled, resolution=1)
    modularity = community.modularity(complete_partition_sampled, G_sampled)
    
    pkl.dump(complete_partition_sampled, open(Path_Communities, "wb"), protocol=4) 
    
    n_comm_10 = len(set(complete_partition_sampled.values()))
    
    community_sizes_10 = Counter(complete_partition_sampled.values())
    
    biggest_community_size_10 = max(community_sizes_10.values())
    
    print(n_nodes, n_edges, n_cc, modularity)
    
    assortativity = nx.degree_assortativity_coefficient(G_sampled)
    
    # Calcula o NMI entre os arquivos pickle
    nmi, intersection = calculate_nmi('whatsapp/original/comm_complete_whatsapp_1.pkl', Path_Communities)
    
    parameter = parameter
    sample = sample
    batch = batch
    
    network = 'WhatsApp'
    type_network = "sample_RW"
    snap = "October"
    
    list_results.append((network, type_network, snap, n_nodes, n_edges, avg_weight, std_weight, avg_degree, std_degree, 
                         density, clustering, std_clustering, n_cc, largest_component_size, n_comm_10,biggest_community_size_10, modularity, assortativity, parameter, sample, batch,intersection, nmi))
    
    df = pd.DataFrame(list_results, columns=['Network', 'Type', 'Snapshot', '# Nodes', '# Edges', 'Avg. Weight', 'Std. Weight',
                                             'Avg. Degree','Std. Degree','Density', 'Avg. Clustering', 'Std. Clustering','# Components',
                                             '# Big. Component Size','# Communities', '# Big. Community Size', 'Modularity', "Assortivity", 
                                             "Parameter","Net. Sample", "Batch", "intersection","nmi"])
    del G_sampled
    # del G
    # del partition
    del complete_partition_sampled

    df.to_csv("whatsapp/RandomWalk/communities/networks_whatsapp_samples_characterization.csv", mode='a', header=header, index=None)

In [ ]:
def characterize_random_walk_sample_networks(graph, dataset):
    header = True
    j = 1
    
    while j <= 10:
        print(j)
        for i in range(1,10,1):
            sample = "whatsapp/RandomWalk/samples/"+dataset+"_"+str(i/10)+'_random_walk_edge_list_batch_'+str(j)+'_.csv'
            Path_Communities = "whatsapp/RandomWalk/communities/"+dataset+"_RW_batch_"+str(j)+"_net_percent_"+str(i/10)+".pkl"
            characterize_sampled_network(sample, Path_Communities, dataset, "sample whatsapp - Random Walk", "sample_"+str(i)+"_batch_"+str(j), 1, 0, header, str(i/10), str(j))
            # print("saved_community_"+dataset+"_"+str(i/10)+'_random_node_edge_list_batch_'+str(j)+'_.csv')
            header = False
        header = False
        j += 1

In [ ]:
# characterize_random_walk_sample_networks(graph, 'whatsapp')

### 7.1 Calculate samples NMI mean and std per parameter

#### In this code I load csv with samples and samples communities info and I group them per parameter and after that I calculate NMI mean and std
#### At the end I store them in a csv for further analysis

In [ ]:
df_random_walk_original_whatsapp = pd.read_csv('RandomWalk/communities/networks_whatsapp_samples_characterization.csv')


In [ ]:
df_RW_NMI_whatsapp = df_random_walk_original_whatsapp

In [ ]:
df_RW_NMI_whatsapp[:3]

In [ ]:
df_RW_NMI_whatsapp["nmi_mean"]=0
df_RW_NMI_whatsapp["std"]=0
df_RW_NMI_whatsapp["nmi_std"] = 0

In [ ]:
df_RW_NMI_whatsapp['nmi_norm'] = df_RW_NMI_whatsapp['nmi'] / 0.4552

In [ ]:
df_RW_NMI_whatsapp['nmi_mean'] = df_RW_NMI_whatsapp.groupby(['Net. Sample'])['nmi_norm'].transform(lambda x: x.mean())

In [ ]:
df_RW_NMI_whatsapp['std'] = df_RW_NMI_whatsapp.groupby(['Net. Sample'])['nmi_norm'].transform(lambda x: x.std(ddof=1))

In [ ]:
df_RW_NMI_whatsapp["nmi_std"] = df_RW_NMI_whatsapp.apply(lambda x: '{:.2f}±{:.2f}'.format(x["nmi_mean"], x["std"]), axis=1)

In [ ]:
new_df_2 = df_RW_NMI_whatsapp[['Batch', 'Net. Sample','nmi_std']].copy()
new_df_2[:3]

In [ ]:
pivoted_2 = new_df_2.pivot(index=["Batch"], columns="Net. Sample")
pd.set_option('display.max_rows', None)
pivoted_2[:100]

In [ ]:
pivoted_2[:1]

In [ ]:
pivoted_2[:1].to_csv("RandomWalk/communities/networks_nmi_whatsapp_samples_normalized.csv")

# Backbone

#### In this section we handle backbone sampling and community detection from our proposed methodology

## 1. Load original network and calculate its backbone

### In our methodology we use two backbone extraction tequiniques according to the dataset reference: Disparity Filter or Gloss Filter

### 1.1 Disparity Filter

##### In this section we present disparity filter used in our approach
##### After apply it in an original network we store the backbone in a csv for further analysis

In [ ]:
def disparity_filter(table, undirected = True, return_self_loops = True):
    sys.stderr.write("Calculating DF score...\n")
    table = table.copy()
    table.reset_index(inplace=True)
    table_sum = table.groupby(table["source"]).sum().reset_index()
    table_deg = table.groupby(table["source"]).count()["target"].reset_index()
    table = table.merge(table_sum, on = "source", how = "left", suffixes = ("", "_sum"))
    table = table.merge(table_deg, on = "source", how = "left", suffixes = ("", "_count"))
    table["score"] = 1.0 - ((1.0 - (table["weight"] / table["weight_sum"])) ** (table["target_count"] - 1))
    table["variance"] = (table["target_count"] ** 2) * (((20 + (4.0 * table["target_count"])) / ((table["target_count"] + 1.0) * (table["target_count"] + 2) * (table["target_count"] + 3))) - ((4.0) / ((table["target_count"] + 1.0) ** 2)))
    if not return_self_loops:
        table = table[table["source"] != table["target"]]
    if undirected:
        table["edge"] = table.apply(lambda x: "%s-%s" % (min(x["source"], x["target"]), max(x["source"], x["target"])), axis = 1)
        table_maxscore = table.groupby(by = "edge")["score"].max().reset_index()
        table_minvar = table.groupby(by = "edge")["variance"].min().reset_index()
        table = table.merge(table_maxscore, on = "edge", suffixes = ("_min", ""))
        table = table.merge(table_minvar, on = "edge", suffixes = ("_max", ""))
        table = table.drop_duplicates(subset = ["edge"])
        table = table.drop("edge", axis=1)
        table = table.drop("score_min", axis=1)
        table = table.drop("variance_max", axis=1)
    return table[["source", "target", "weight", "score", "variance"]]


def read(filename, column_of_interest, triangular_input = False, 
         consider_self_loops = True, undirected = True, drop_zeroes = True, sep = ","):

    table = pd.read_csv(filename, sep = sep)
    table = table[["source", "target", column_of_interest]]
    table.rename(columns = {column_of_interest: "weight"}, inplace = True)
    if drop_zeroes:
        table = table[table["weight"] > 0]
    if not consider_self_loops:
        table = table[table["source"] != table["target"]]
    if triangular_input:
        table2 = table.copy()
        table2["new_source"] = table["target"]
        table2["new_target"] = table["source"]
        table2.drop("source", axis=1, inplace = True)
        table2.drop("target", axis=1, inplace = True)
        table2 = table2.rename(columns = {"new_source": "source", "new_target": "target"})
        table = pd.concat([table, table2], axis = 0)
        table = table.drop_duplicates(subset = ["source", "target"])
    original_nodes = len(set(table["source"]) | set(table["target"]))
    original_edges = table.shape[0]
    if undirected:
        return table
    else:
        return table

In [ ]:
df_edges = read('OctoberWpp.edgelist', column_of_interest='weight',
                triangular_input=True,
                consider_self_loops=False,
                undirected=True, drop_zeroes=True)

In [ ]:
df_edges[:3]

In [ ]:
confidence = 0.90
nc_table = disparity_filter(df_edges, undirected = True)
nc_table = nc_table[nc_table['score'] >= confidence]
print(nc_table[:3])

In [ ]:
backbone_file_csv = nc_table.iloc[:, :3]
backbone_file_csv = backbone_file_csv.reset_index(drop=True)

In [ ]:
backbone_file_csv[:3]

In [ ]:
nc_table = nc_table.reset_index(drop=True)

In [ ]:
nc_table[:3]

In [ ]:
nc_table.to_csv("whatsapp/backbone/backbone_complete.csv",index=False)
backbone_file_csv.to_csv("whatsapp/backbone/backbone_wpp.edgelist",index=False, header=False)

## 2. Calculate the backbone of each sample and store it in a csv

In [ ]:
def backbone_random_node_samples(num_repeticoes, amostragem):
    dataset = 'whatsapp'
    excluded_numbers = []
    j = 1
    for _ in range(num_repeticoes):
        
        file = "whatsapp/RandomNode/samples/"+dataset+"_"+str(amostragem)+'_random_node_edge_list_batch_'+str(j)+'_.csv'
        df_edges = read(file, column_of_interest='weight',
                triangular_input=True,
                consider_self_loops=False,
                undirected=True, drop_zeroes=True)

        nc_table = disparity_filter(df_edges)
        nc_table = nc_table.reset_index(drop=True)
        nc_table.to_csv("whatsapp/RandomNode/backbone_samples_complete/"+dataset+"_"+str(amostragem)+'_random_node_edge_list_batch_'+str(j)+'_.csv',index=False)
        
        
        print(nc_table[:3])
        
        
        sample = nc_table.iloc[:, :3]
        print(sample[:3])
        sample= sample.reset_index(drop=True)
        sample.to_csv("whatsapp/RandomNode/backbone_samples/"+dataset+"_"+str(amostragem)+'_random_node_edge_list_batch_'+str(j)+'_.csv',index=False)

        j+=1

In [ ]:
def backbone_random_walk_samples(num_repeticoes, amostragem):
    dataset = 'whatsapp'
    excluded_numbers = []
    j = 1
    for _ in range(num_repeticoes):
        
        file = "whatsapp/RandomWalk/samples/"+dataset+"_"+str(amostragem)+'_random_walk_edge_list_batch_'+str(j)+'_.csv'
        df_edges = read(file, column_of_interest='weight',
                triangular_input=True,
                consider_self_loops=False,
                undirected=True, drop_zeroes=True)

        nc_table = disparity_filter(df_edges)
        nc_table = nc_table.reset_index(drop=True)
        nc_table.to_csv("whatsapp/RandomWalk/backbone_samples_complete/"+dataset+"_"+str(amostragem)+'_random_walk_edge_list_batch_'+str(j)+'_.csv',index=False)
        
        
        print(nc_table[:3])
        
        
        sample = nc_table.iloc[:, :3]
        print(sample[:3])
        sample= sample.reset_index(drop=True)
        sample.to_csv("whatsapp/RandomWalk/backbone_samples/"+dataset+"_"+str(amostragem)+'_random_walk_edge_list_batch_'+str(j)+'_.csv',index=False)

        j+=1

In [ ]:
def backbone_forest_fire_samples():
    # Inicializa uma lista para armazenar os resultados do NMI
    dataset = 'whatsapp'
    excluded_numbers = []
    k = 1
    # Realiza a amostragem e o cálculo do NMI várias vezes
    while k <= 10:
        print(k)
        for i in range(1,11,1):
            values_seed = []
            for j in range(1,10,1):
                file = "whatsapp/ForestFire/samples/"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch'+str(k)+'_'+str(i)+'.csv'
                df_edges = read(file, column_of_interest='weight',
                triangular_input=True,
                consider_self_loops=False,
                undirected=True, drop_zeroes=True)
                nc_table = disparity_filter(df_edges)
                nc_table = nc_table.reset_index(drop=True)
                nc_table.to_csv("whatsapp/ForestFire/backbone_samples_complete/"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch'+str(k)+'_'+str(i)+'.csv',index=False)
                sample = nc_table.iloc[:, :3]
                sample.to_csv("whatsapp/ForestFire/backbone_samples/"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch'+str(k)+'_'+str(i)+'.csv',index=False)
        k += 1

## 3. Execute 10x community detection in backbone of the original network

##### I store in a csv to have a backup for further analysis

In [ ]:
edgelist_path = 'whatsapp/backbone/backbone_wpp.edgelist'
G = nx.read_weighted_edgelist(edgelist_path, delimiter=",")


In [ ]:
graph = G
number_of_nodes = graph.number_of_nodes()
print(number_of_nodes)
print(graph.number_of_edges())
print(graph.is_directed())
# print(nx.is_connected(graph))
is_connected = nx.is_connected(graph)

In [ ]:
def characterize_network_executions(G, Path_Communities, header,index):
    list_results = []
                
    n_nodes = len(G.nodes())
    n_edges = len(G.edges())
    
    degree = dict(G.degree)
    avg_degree = round(np.average(list(degree.values())),4)
    std_degree = round(np.std(list(degree.values())),4)
    
    edge_weights = nx.get_edge_attributes(G, 'weight')
    edge_weights_values = list(edge_weights.values())
    avg_weight = np.mean(edge_weights_values)
    std_weight = np.std(edge_weights_values)
    
    density = round(nx.density(G),4)
    
    n_cc = nx.number_connected_components(G)
    components = list(nx.connected_components(G))
    component_sizes = [len(component) for component in components]
    largest_component_size = max(component_sizes)
    
    clustering = round(nx.average_clustering(G),4)
    clustering_coeffs = nx.clustering(G)
    clustering_coeffs_values = list(clustering_coeffs.values())
    std_clustering = np.std(clustering_coeffs_values)
    
    # Calculate Louvain community detection
    partition = community.best_partition(G, resolution=1)
    modularity = community.modularity(partition, G)
    
    partition = {node: comm_id for node, comm_id in partition.items() if list(partition.values()).count(comm_id) > 10}
    
    pkl.dump(partition, open(Path_Communities, "wb"), protocol=4)
    
    n_comm = len(set(partition.values()))
    community_sizes = Counter(partition.values())
    biggest_community_size = max(community_sizes.values())
    
    
    print(n_nodes, n_edges, n_cc, modularity)
    
    assortativity = nx.degree_assortativity_coefficient(G)
    
    parameter = '-'
    sample = "-"
    batch = "-"
    intersection = "-"
    nmi = "-"
    
    network = 'WhatsApp'
    type_network = "original_"+str(i)
    snap = "October"
    
    list_results.append((network, type_network, snap, n_nodes, n_edges, avg_weight, std_weight, avg_degree, std_degree, 
                         density, clustering, std_clustering, n_cc, largest_component_size, n_comm,biggest_community_size, modularity, assortativity, parameter, sample, batch,intersection, nmi))
    
    df = pd.DataFrame(list_results, columns=['Network', 'Type', 'Snapshot', '# Nodes', '# Edges', 'Avg. Weight', 'Std. Weight',
                                             'Avg. Degree','Std. Degree','Density', 'Avg. Clustering', 'Std. Clustering','# Components',
                                             '# Big. Component Size','# Communities', '# Big. Community Size', 'Modularity', "Assortivity", 
                                             "Parameter","Net. Sample", "Batch", "intersection","nmi"])

    df.to_csv("whatsapp/backbone/networks_whatsapp_original_characterization.csv", mode='a', header=header, index=None)

In [ ]:
header = True
for i in range(1, 11):
    characterize_network_executions(graph, 'whatsapp/backbone/comm_complete_whatsapp_'+str(i)+'.pkl', header, i)
    header = False

### 3.1 Compare communities of the executions and calculate NMI mean and std

In [ ]:
min_tamanho_comunidade = 10
PATH_Communities = 'whatsapp/backbone/'
list_results = []
list_models = ['comm_complete_whatsapp_1', 'comm_complete_whatsapp_2', 
               'comm_complete_whatsapp_3', 'comm_complete_whatsapp_4', 
               'comm_complete_whatsapp_5', 'comm_complete_whatsapp_6', 
               'comm_complete_whatsapp_7', 'comm_complete_whatsapp_8',
               'comm_complete_whatsapp_9', 'comm_complete_whatsapp_10']

for i in range(1,len(list_models),1):
    model_1 = list_models[0]
    model_2 = list_models[i]
  
    # Load the pickled communities
    with open(PATH_Communities + model_1 + '.pkl', 'rb') as file:
        dicionario1 = pickle.load(file)
    with open(PATH_Communities + model_2 + '.pkl', 'rb') as file:
        dicionario2 = pickle.load(file)

    # Contagem do tamanho das comunidades e filtragem
    comunidades_filtradas1 = {}
    comunidades_filtradas2 = {}

    # Filtra as comunidades com tamanho maior ou igual a min_tamanho_comunidade
    for node, community_id in dicionario1.items():
        tamanho_comunidade = list(dicionario1.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas1[node] = community_id

    for node, community_id in dicionario2.items():
        tamanho_comunidade = list(dicionario2.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas2[node] = community_id

    # Obter as chaves em comum entre os dicionários filtrados
    chaves_comuns = set(comunidades_filtradas1.keys()) & set(comunidades_filtradas2.keys())
    intersection = len(chaves_comuns)
    # print(intersection)
    # Inicializar as listas para armazenar os valores correspondentes
    valores_dicionario1 = []
    valores_dicionario2 = []

    # Preencher as listas com os valores correspondentes, mantendo a ordem
    for chave in chaves_comuns:
        valores_dicionario1.append(comunidades_filtradas1[chave])
        valores_dicionario2.append(comunidades_filtradas2[chave])

    nmi = 0
    # Calcular o NMI usando scikit-learn
    if(intersection> 1):
        nmi = normalized_mutual_info_score(valores_dicionario1, valores_dicionario2)

    list_results.append((model_1, model_2, nmi))

df = pd.DataFrame(list_results, columns = ['Model 1', 'Model 2', 'NMI'])

# Save the DataFrame as a CSV file
csv_file = 'whatsapp/backbone/nmi_original_whatsapp_results.csv'
df.to_csv(csv_file, index=False)

df

mean_nmi = df["NMI"].mean()
std_nmi = df["NMI"].std()

# Create a new DataFrame with the mean and std values
result_df = pd.DataFrame({"NMI": ['{:.4f}±{:.4f}'.format(mean_nmi,std_nmi)]})

print(result_df)

csv_file = 'whatsapp/backbone/nmi_mean_whatsapp_results.csv'
result_df.to_csv(csv_file, index=False)

## 4. Detect community of backbones of the generated samples networks - Random Node

##### In this scenario I have created a separeted method called characterize_random_node_sample_networks_backbone_rn to call characterize_sampled_network_backbone_rn to detect the  communities of each sample and, after that, I call in it a separated method calculate_nmi to calcutate NMI of the samples.
##### I also store the communities info in a separated files for further analysis and I put all the backbone samples info in another csv.

In [ ]:
def calculate_nmi(dicionario1_pkl, dicionario2_pkl, min_tamanho_comunidade=10):
    #from sklearn.metrics.cluster import adjusted_mutual_info_score
    # Carregar os dicionários a partir dos arquivos pickle
    with open(dicionario1_pkl, 'rb') as arquivo1:
        dict1 = pkl.load(arquivo1)
    
    with open(dicionario2_pkl, 'rb') as arquivo2:
        dict2 = pkl.load(arquivo2)
        
    dicionario1 = {int(key): value for key, value in dict1.items()}
    dicionario2 = {int(key): value for key, value in dict2.items()}

    # Contagem do tamanho das comunidades e filtragem
    comunidades_filtradas1 = {}
    comunidades_filtradas2 = {}
    
    # Filtra as comunidades com tamanho maior ou igual a min_tamanho_comunidade
    for node, community_id in dicionario1.items():
        tamanho_comunidade = list(dicionario1.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas1[node] = community_id

    for node, community_id in dicionario2.items():
        tamanho_comunidade = list(dicionario2.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas2[node] = community_id
    
    chaves_comuns = list(set(comunidades_filtradas1.keys()).intersection(comunidades_filtradas2.keys()))
    
    intersection = len(chaves_comuns)
    print(intersection)
    # Inicializar as listas para armazenar os valores correspondentes
    valores_dicionario1 = []
    valores_dicionario2 = []
    
    # Preencher as listas com os valores correspondentes, mantendo a ordem
    for chave in chaves_comuns:
        valores_dicionario1.append(comunidades_filtradas1[chave])
        valores_dicionario2.append(comunidades_filtradas2[chave])
    
    if(intersection <= 1): return 0, 0

    # Calcular o NMI usando scikit-learn
    nmi = normalized_mutual_info_score(valores_dicionario1, valores_dicionario2)
    print(nmi)
    
    return nmi, intersection

In [ ]:
def characterize_sampled_network_backbone_rn(graph, Path_Communities, network, type_network, snap, s, parameter, header, sample, batch):
    #Para rede amostrada
    list_results = []
    print(graph)
    original_df = pd.read_csv(graph)
    print(original_df[:3])
    filtered_df = original_df[original_df['score'] > 0.90]
    print(filtered_df[:3])
    df = filtered_df.iloc[:, :3]
    print(df[:3])
    Graphtype = nx.Graph()
    G_sampled = nx.from_pandas_edgelist(df, edge_attr='weight', create_using=Graphtype)
    
    # for component in list(nx.connected_components(G_sampled)):
    #     if len(component)<3:
    #         for node in component:
    #             G_sampled.remove_node(node)
                
    n_nodes = len(G_sampled.nodes())
    n_edges = len(G_sampled.edges())
    
    degree = dict(G_sampled.degree)
    avg_degree = round(np.average(list(degree.values())),4)
    std_degree = round(np.std(list(degree.values())),4)
    
    edge_weights = nx.get_edge_attributes(G_sampled, 'weight')
    edge_weights_values = list(edge_weights.values())
    avg_weight = np.mean(edge_weights_values)
    std_weight = np.std(edge_weights_values)
    
    density = round(nx.density(G_sampled),4)
    
    n_cc = nx.number_connected_components(G_sampled)
    components = list(nx.connected_components(G_sampled))
    component_sizes = [len(component) for component in components]
    largest_component_size = max(component_sizes)
    
    clustering = round(nx.average_clustering(G_sampled),4)
    clustering_coeffs = nx.clustering(G_sampled)
    clustering_coeffs_values = list(clustering_coeffs.values())
    std_clustering = np.std(clustering_coeffs_values)
    
    
    complete_partition_sampled = community.best_partition(G_sampled, resolution=1)
    modularity = community.modularity(complete_partition_sampled, G_sampled)
    
    pkl.dump(complete_partition_sampled, open(Path_Communities, "wb"), protocol=4) 
    
    n_comm_10 = len(set(complete_partition_sampled.values()))
    
    community_sizes_10 = Counter(complete_partition_sampled.values())
    
    biggest_community_size_10 = max(community_sizes_10.values())
    
    print(n_nodes, n_edges, n_cc, modularity)
    
    assortativity = nx.degree_assortativity_coefficient(G_sampled)
    
    # Calcula o NMI entre os arquivos pickle
    nmi, intersection = calculate_nmi('whatsapp/backbone/comm_complete_whatsapp_1.pkl', Path_Communities)
    
    parameter = parameter
    sample = sample
    batch = batch
    
    network = 'WhatsApp'
    type_network = "sample_RN"
    snap = "October"
    
    list_results.append((network, type_network, snap, n_nodes, n_edges, avg_weight, std_weight, avg_degree, std_degree, 
                         density, clustering, std_clustering, n_cc, largest_component_size, n_comm_10,biggest_community_size_10, modularity, assortativity, parameter, sample, batch,intersection, nmi))
    
    df = pd.DataFrame(list_results, columns=['Network', 'Type', 'Snapshot', '# Nodes', '# Edges', 'Avg. Weight', 'Std. Weight',
                                             'Avg. Degree','Std. Degree','Density', 'Avg. Clustering', 'Std. Clustering','# Components',
                                             '# Big. Component Size','# Communities', '# Big. Community Size', 'Modularity', "Assortivity", 
                                             "Parameter","Net. Sample", "Batch", "intersection","nmi"])
    del G_sampled
    # del G
    # del partition
    del complete_partition_sampled

    df.to_csv("whatsapp/RandomNode/communities_backbone/networks_whatsapp_samples_characterization.csv", mode='a', header=header, index=None)

In [ ]:
def characterize_random_node_sample_networks_backbone_rn(graph, dataset):
    header = True
    j = 1
    
    while j <= 10:
        print(j)
        for i in range(1,11,1):
            sample = "whatsapp/RandomNode/backbone_samples_complete/"+dataset+"_"+str(i/10)+'_random_node_edge_list_batch_'+str(j)+'_.csv'
            Path_Communities = "whatsapp/RandomNode/communities_backbone/"+dataset+"_RN_batch_"+str(j)+"_net_percent_"+str(i/10)+".pkl"
            characterize_sampled_network_backbone_rn(sample, Path_Communities, dataset, "sample whatsapp - Random Node", "sample_"+str(i)+"_batch_"+str(j), 1, 0, header, str(i/10), str(j))
            # print("saved_community_"+dataset+"_"+str(i/10)+'_random_node_edge_list_batch_'+str(j)+'_.csv')
            header = False
        j += 1

In [ ]:
# characterize_random_node_sample_networks_backbone_rn(graph, 'whatsapp')

### 4.1 Calculate samples NMI mean and std per parameter

#### In this code I load csv with samples and samples communities info and I group them per parameter and after that I calculate NMI mean and std
#### At the end I store them in a csv for further analysis

In [ ]:
del df_random_node_original_whatsapp, df_RN_NMI_whatsapp

In [ ]:
df_random_node_original_whatsapp = pd.read_csv('RandomNode/communities_backbone/networks_whatsapp_samples_characterization.csv')


In [ ]:
df_RN_NMI_whatsapp = df_random_node_original_whatsapp

In [ ]:
df_RN_NMI_whatsapp[:3]

In [ ]:
df_RN_NMI_whatsapp["nmi_mean"]=0
df_RN_NMI_whatsapp["std"]=0
df_RN_NMI_whatsapp["nmi_std"] = 0

In [ ]:
df_RN_NMI_whatsapp['nmi_norm'] = df_RN_NMI_whatsapp['nmi'] / 0.5875

In [ ]:
df_RN_NMI_whatsapp['nmi_mean'] = df_RN_NMI_whatsapp.groupby(['Net. Sample'])['nmi_norm'].transform(lambda x: x.mean())

In [ ]:
df_RN_NMI_whatsapp['std'] = df_RN_NMI_whatsapp.groupby(['Net. Sample'])['nmi_norm'].transform(lambda x: x.std(ddof=1))

In [ ]:
df_RN_NMI_whatsapp["nmi_std"] = df_RN_NMI_whatsapp.apply(lambda x: '{:.2f}±{:.2f}'.format(x["nmi_mean"], x["std"]), axis=1)

In [ ]:
new_df_2 = df_RN_NMI_whatsapp[['Batch', 'Net. Sample','nmi_std']].copy()
new_df_2[:3]

In [ ]:
pivoted_2 = new_df_2.pivot(index=["Batch"], columns="Net. Sample")
pd.set_option('display.max_rows', None)
pivoted_2[:100]

In [ ]:
pivoted_2[:1]

In [ ]:
pivoted_2[:1].to_csv("RandomNode/communities_backbone/networks_nmi_whatsapp_samples_normalized.csv")

## 5. Detect community of backbones of the generated samples networks - Random Walk

##### In this scenario I have created a separeted method called characterize_random_node_sample_networks_backbone_rw to call characterize_sampled_network_backbone_rw to detect the  communities of each sample and, after that, I call in it a separated method calculate_nmi to calcutate NMI of the samples.
##### I also store the communities info in a separated files for further analysis and I put all the backbone samples info in another csv.

In [ ]:
def calculate_nmi(dicionario1_pkl, dicionario2_pkl, min_tamanho_comunidade=10):
    #from sklearn.metrics.cluster import adjusted_mutual_info_score
    # Carregar os dicionários a partir dos arquivos pickle
    with open(dicionario1_pkl, 'rb') as arquivo1:
        dict1 = pkl.load(arquivo1)
    
    with open(dicionario2_pkl, 'rb') as arquivo2:
        dict2 = pkl.load(arquivo2)
        
    dicionario1 = {int(key): value for key, value in dict1.items()}
    dicionario2 = {int(key): value for key, value in dict2.items()}

    # Contagem do tamanho das comunidades e filtragem
    comunidades_filtradas1 = {}
    comunidades_filtradas2 = {}
    
    # Filtra as comunidades com tamanho maior ou igual a min_tamanho_comunidade
    for node, community_id in dicionario1.items():
        tamanho_comunidade = list(dicionario1.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas1[node] = community_id

    for node, community_id in dicionario2.items():
        tamanho_comunidade = list(dicionario2.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas2[node] = community_id
    
    chaves_comuns = list(set(comunidades_filtradas1.keys()).intersection(comunidades_filtradas2.keys()))
    
    intersection = len(chaves_comuns)
    print(intersection)
    # Inicializar as listas para armazenar os valores correspondentes
    valores_dicionario1 = []
    valores_dicionario2 = []
    
    # Preencher as listas com os valores correspondentes, mantendo a ordem
    for chave in chaves_comuns:
        valores_dicionario1.append(comunidades_filtradas1[chave])
        valores_dicionario2.append(comunidades_filtradas2[chave])
    
    if(intersection <= 1): return 0, 0

    # Calcular o NMI usando scikit-learn
    nmi = normalized_mutual_info_score(valores_dicionario1, valores_dicionario2)
    print(nmi)
    
    return nmi, intersection

In [ ]:
def characterize_sampled_network_backbone_rw(graph, Path_Communities, network, type_network, snap, s, parameter, header, sample, batch):
    #Para rede amostrada
    list_results = []
    print(graph)
    original_df = pd.read_csv(graph)
    print(original_df[:3])
    filtered_df = original_df[original_df['score'] > 0.90]
    print(filtered_df[:3])
    df = filtered_df.iloc[:, :3]
    print(df[:3])
    Graphtype = nx.Graph()
    G_sampled = nx.from_pandas_edgelist(df, edge_attr='weight', create_using=Graphtype)
    
    # for component in list(nx.connected_components(G_sampled)):
    #     if len(component)<3:
    #         for node in component:
    #             G_sampled.remove_node(node)
                
    n_nodes = len(G_sampled.nodes())
    n_edges = len(G_sampled.edges())
    
    degree = dict(G_sampled.degree)
    avg_degree = round(np.average(list(degree.values())),4)
    std_degree = round(np.std(list(degree.values())),4)
    
    edge_weights = nx.get_edge_attributes(G_sampled, 'weight')
    edge_weights_values = list(edge_weights.values())
    avg_weight = np.mean(edge_weights_values)
    std_weight = np.std(edge_weights_values)
    
    density = round(nx.density(G_sampled),4)
    
    n_cc = nx.number_connected_components(G_sampled)
    components = list(nx.connected_components(G_sampled))
    component_sizes = [len(component) for component in components]
    largest_component_size = max(component_sizes)
    
    clustering = round(nx.average_clustering(G_sampled),4)
    clustering_coeffs = nx.clustering(G_sampled)
    clustering_coeffs_values = list(clustering_coeffs.values())
    std_clustering = np.std(clustering_coeffs_values)
    
    
    complete_partition_sampled = community.best_partition(G_sampled, resolution=1)
    modularity = community.modularity(complete_partition_sampled, G_sampled)
    
    pkl.dump(complete_partition_sampled, open(Path_Communities, "wb"), protocol=4) 
    
    n_comm_10 = len(set(complete_partition_sampled.values()))
    
    community_sizes_10 = Counter(complete_partition_sampled.values())
    
    biggest_community_size_10 = max(community_sizes_10.values())
    
    print(n_nodes, n_edges, n_cc, modularity)
    
    assortativity = nx.degree_assortativity_coefficient(G_sampled)
    
    # Calcula o NMI entre os arquivos pickle
    nmi, intersection = calculate_nmi('whatsapp/backbone/comm_complete_whatsapp_1.pkl', Path_Communities)
    
    parameter = parameter
    sample = sample
    batch = batch
    
    network = 'WhatsApp'
    type_network = "sample_RW"
    snap = "October"
    
    list_results.append((network, type_network, snap, n_nodes, n_edges, avg_weight, std_weight, avg_degree, std_degree, 
                         density, clustering, std_clustering, n_cc, largest_component_size, n_comm_10,biggest_community_size_10, modularity, assortativity, parameter, sample, batch,intersection, nmi))
    
    df = pd.DataFrame(list_results, columns=['Network', 'Type', 'Snapshot', '# Nodes', '# Edges', 'Avg. Weight', 'Std. Weight',
                                             'Avg. Degree','Std. Degree','Density', 'Avg. Clustering', 'Std. Clustering','# Components',
                                             '# Big. Component Size','# Communities', '# Big. Community Size', 'Modularity', "Assortivity", 
                                             "Parameter","Net. Sample", "Batch", "intersection","nmi"])
    del G_sampled
    # del G
    # del partition
    del complete_partition_sampled

    df.to_csv("whatsapp/RandomWalk/communities_backbone/networks_whatsapp_samples_characterization.csv", mode='a', header=header, index=None)

In [ ]:
def characterize_random_node_sample_networks_backbone_rw(graph, dataset):
    header = True
    j = 1
    
    while j <= 10:
        print(j)
        for i in range(1,10,1):
            sample = "whatsapp/RandomWalk/backbone_samples_complete/"+dataset+"_"+str(i/10)+'_random_walk_edge_list_batch_'+str(j)+'_.csv'
            Path_Communities = "whatsapp/RandomWalk/communities_backbone/"+dataset+"_RW_batch_"+str(j)+"_net_percent_"+str(i/10)+".pkl"
            characterize_sampled_network_backbone_rw(sample, Path_Communities, dataset, "sample whatsapp - Random Walk", "sample_"+str(i)+"_batch_"+str(j), 1, 0, header, str(i/10), str(j))
            # print("saved_community_"+dataset+"_"+str(i/10)+'_random_node_edge_list_batch_'+str(j)+'_.csv')
            header = False
        j += 1

In [ ]:
# characterize_random_node_sample_networks_backbone_rw(graph, 'whatsapp')

### 5.1 Calculate samples NMI mean and std per parameter

#### In this code I load csv with samples and samples communities info and I group them per parameter and after that I calculate NMI mean and std
#### At the end I store them in a csv for further analysis

In [ ]:
df_random_node_original_whatsapp = pd.read_csv('RandomWalk/communities_backbone/networks_whatsapp_samples_characterization.csv')


In [ ]:
df_RN_NMI_whatsapp = df_random_node_original_whatsapp

In [ ]:
df_RN_NMI_whatsapp[:3]

In [ ]:
df_RN_NMI_whatsapp["nmi_mean"]=0
df_RN_NMI_whatsapp["std"]=0
df_RN_NMI_whatsapp["nmi_std"] = 0

In [ ]:
df_RN_NMI_whatsapp['nmi_norm'] = df_RN_NMI_whatsapp['nmi'] / 0.5875

In [ ]:
df_RN_NMI_whatsapp['nmi_mean'] = df_RN_NMI_whatsapp.groupby(['Net. Sample'])['nmi_norm'].transform(lambda x: x.mean())

In [ ]:
df_RN_NMI_whatsapp['std'] = df_RN_NMI_whatsapp.groupby(['Net. Sample'])['nmi_norm'].transform(lambda x: x.std(ddof=1))

In [ ]:
df_RN_NMI_whatsapp["nmi_std"] = df_RN_NMI_whatsapp.apply(lambda x: '{:.2f}±{:.2f}'.format(x["nmi_mean"], x["std"]), axis=1)

In [ ]:
new_df_2 = df_RN_NMI_whatsapp[['Batch', 'Net. Sample','nmi_std']].copy()
new_df_2[:3]

In [ ]:
pivoted_2 = new_df_2.pivot(index=["Batch"], columns="Net. Sample")
pd.set_option('display.max_rows', None)
pivoted_2[:100]

In [ ]:
pivoted_2[:1]

In [ ]:
pivoted_2[:1].to_csv("RandomWalk/communities_backbone/networks_nmi_whatsapp_samples_backbone.csv")

## 6. Detect community of backbones of the generated samples networks - Forest Fire

##### In this scenario I have created a separeted method called characterize_forest_fire_sample_networks_backbone to call characterize_sampled_network_backbone_ff to detect the  communities of each sample and, after that, I call in it a separated method calculate_nmi to calcutate NMI of the samples.
##### I also store the communities info in a separated files for further analysis and I put all the backbone samples info in another csv.

In [ ]:
def calculate_nmi(dicionario1_pkl, dicionario2_pkl, min_tamanho_comunidade=10):
    #from sklearn.metrics.cluster import adjusted_mutual_info_score
    # Carregar os dicionários a partir dos arquivos pickle
    with open(dicionario1_pkl, 'rb') as arquivo1:
        dict1 = pkl.load(arquivo1)
    
    with open(dicionario2_pkl, 'rb') as arquivo2:
        dict2 = pkl.load(arquivo2)
        
    dicionario1 = {int(key): value for key, value in dict1.items()}
    dicionario2 = {int(key): value for key, value in dict2.items()}

    # Contagem do tamanho das comunidades e filtragem
    comunidades_filtradas1 = {}
    comunidades_filtradas2 = {}
    
    # Filtra as comunidades com tamanho maior ou igual a min_tamanho_comunidade
    for node, community_id in dicionario1.items():
        tamanho_comunidade = list(dicionario1.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas1[node] = community_id

    for node, community_id in dicionario2.items():
        tamanho_comunidade = list(dicionario2.values()).count(community_id)
        if tamanho_comunidade >= min_tamanho_comunidade:
            comunidades_filtradas2[node] = community_id
    
    chaves_comuns = list(set(comunidades_filtradas1.keys()).intersection(comunidades_filtradas2.keys()))
    
    intersection = len(chaves_comuns)
    print(intersection)
    # Inicializar as listas para armazenar os valores correspondentes
    valores_dicionario1 = []
    valores_dicionario2 = []
    
    # Preencher as listas com os valores correspondentes, mantendo a ordem
    for chave in chaves_comuns:
        valores_dicionario1.append(comunidades_filtradas1[chave])
        valores_dicionario2.append(comunidades_filtradas2[chave])
    
    if(intersection <= 1): return 0, 0

    # Calcular o NMI usando scikit-learn
    nmi = normalized_mutual_info_score(valores_dicionario1, valores_dicionario2)
    print(nmi)
    
    return nmi, intersection

In [ ]:
def characterize_sampled_network_backbone_ff(graph, Path_Communities, network, type_network, snap, s, parameter, header, sample, batch):
    #Para rede amostrada
    list_results = []
    print(graph)
    original_df = pd.read_csv(graph)
    print(original_df[:3])
    filtered_df = original_df[original_df['score'] > 0.90]
    print(filtered_df[:3])
    df = filtered_df.iloc[:, :3]
    print(df[:3])
    Graphtype = nx.Graph()
    G_sampled = nx.from_pandas_edgelist(df, edge_attr='weight', create_using=Graphtype)
    
    # for component in list(nx.connected_components(G_sampled)):
    #     if len(component)<3:
    #         for node in component:
    #             G_sampled.remove_node(node)
                
    n_nodes = len(G_sampled.nodes())
    n_edges = len(G_sampled.edges())
    
    degree = dict(G_sampled.degree)
    avg_degree = round(np.average(list(degree.values())),4)
    std_degree = round(np.std(list(degree.values())),4)
    
    edge_weights = nx.get_edge_attributes(G_sampled, 'weight')
    edge_weights_values = list(edge_weights.values())
    avg_weight = np.mean(edge_weights_values)
    std_weight = np.std(edge_weights_values)
    
    density = round(nx.density(G_sampled),4)
    
    n_cc = nx.number_connected_components(G_sampled)
    components = list(nx.connected_components(G_sampled))
    component_sizes = [len(component) for component in components]
    largest_component_size = max(component_sizes)
    
    clustering = round(nx.average_clustering(G_sampled),4)
    clustering_coeffs = nx.clustering(G_sampled)
    clustering_coeffs_values = list(clustering_coeffs.values())
    std_clustering = np.std(clustering_coeffs_values)
    
    
    complete_partition_sampled = community.best_partition(G_sampled, resolution=1)
    modularity = community.modularity(complete_partition_sampled, G_sampled)
    
    pkl.dump(complete_partition_sampled, open(Path_Communities, "wb"), protocol=4) 
    
    n_comm_10 = len(set(complete_partition_sampled.values()))
    
    community_sizes_10 = Counter(complete_partition_sampled.values())
    
    biggest_community_size_10 = max(community_sizes_10.values())
    
    print(n_nodes, n_edges, n_cc, modularity)
    
    assortativity = nx.degree_assortativity_coefficient(G_sampled)
    
    # Calcula o NMI entre os arquivos pickle
    nmi, intersection = calculate_nmi('whatsapp/backbone/comm_complete_whatsapp_1.pkl', Path_Communities)
    
    parameter = parameter
    sample = sample
    batch = batch
    
    network = 'WhatsApp'
    type_network = "sample_FF"
    snap = "October"
    
    list_results.append((network, type_network, snap, n_nodes, n_edges, avg_weight, std_weight, avg_degree, std_degree, 
                         density, clustering, std_clustering, n_cc, largest_component_size, n_comm_10,biggest_community_size_10, modularity, assortativity, parameter, sample, batch,intersection, nmi))
    
    df = pd.DataFrame(list_results, columns=['Network', 'Type', 'Snapshot', '# Nodes', '# Edges', 'Avg. Weight', 'Std. Weight',
                                             'Avg. Degree','Std. Degree','Density', 'Avg. Clustering', 'Std. Clustering','# Components',
                                             '# Big. Component Size','# Communities', '# Big. Community Size', 'Modularity', "Assortivity", 
                                             "Parameter","Net. Sample", "Batch", "intersection","nmi"])
    del G_sampled
    # del G
    # del partition
    del complete_partition_sampled

    df.to_csv("whatsapp/ForestFire/communities_backbone/networks_whatsapp_samples_characterization.csv", mode='a', header=header, index=None)

In [ ]:
def characterize_forest_fire_sample_networks_backbone(dataset):
    header = True
    k = 1
    while k <= 10:
        print(k)
        for i in range(1,11,1):
            for j in range(1,10,1):
                sample = "whatsapp/ForestFire/backbone_samples_complete/"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch'+str(k)+'_'+str(i)+'.csv'
                Path_Communities = "whatsapp/ForestFire/communities_backbone/"+dataset+"_FF_batch_"+str(k)+"_net_percent_"+str(i/10)+"_probability_"+str(j/10)+".pkl"
                characterize_sampled_network_backbone_ff(sample, Path_Communities, dataset, "sample Whatsapp - Forest Fire", "batch_"+str(k)+'_sample_'+str(i/10)+'_probaability_'+str(j/10), 1, str(j/10), header, str(i/10), str(k))
                header = False
            # print("saved_community_"+dataset+"_"+str(i/10)+"_probability_"+str(j/10)+'_forest_fire_edge_list_batch_'+str(k)+'_'+str(i)+'_.csv')
            header = False
        k += 1

In [ ]:
# characterize_forest_fire_sample_networks_backbone('whatsapp')

### 6.1 Calculate samples NMI mean and std per parameter

#### In this code I load csv with samples and samples communities info and I group them per parameter and after that I calculate NMI mean and std
#### At the end I store them in a csv for further analysis

In [ ]:
df_forest_fire_original_whatsapp = pd.read_csv('ForestFire/communities_backbone/networks_whatsapp_samples_characterization.csv')


In [ ]:
df_FF_NMI_whatsapp = df_forest_fire_original_whatsapp

In [ ]:
df_FF_NMI_whatsapp[:3]

In [ ]:
df_FF_NMI_whatsapp["nmi_mean"]=0
df_FF_NMI_whatsapp["std"]=0
df_FF_NMI_whatsapp["nmi_std"] = 0

In [ ]:
df_FF_NMI_whatsapp['nmi_norm'] = df_FF_NMI_whatsapp['nmi'] / 0.5875

In [ ]:
df_FF_NMI_whatsapp['nmi_mean'] = df_FF_NMI_whatsapp.groupby(['Net. Sample','Parameter'])['nmi_norm'].transform(lambda x: x.mean())

In [ ]:
df_FF_NMI_whatsapp['std_1'] = df_FF_NMI_whatsapp.groupby(['Net. Sample','Parameter'])['nmi_norm'].transform(lambda x: x.std(ddof=1))

In [ ]:
df_FF_NMI_whatsapp["nmi_std"] = df_FF_NMI_whatsapp.apply(lambda x: '{:.2f}±{:.2f}'.format(x["nmi_mean"], x["std_1"]), axis=1)

In [ ]:
new_df_2 = df_FF_NMI_whatsapp[['Batch','Net. Sample', 'Parameter','nmi_std']].copy()
new_df_2[:3]

In [ ]:
rslt_df_2 = new_df_2[new_df_2['Batch'] == 1]
rslt_df_2.drop('Batch', axis=1, inplace=True)

In [ ]:
pivoted_2 = rslt_df_2.pivot(index=["Net. Sample"], columns=["Parameter"])
pd.set_option('display.max_rows', None)
pivoted_2[:100]

In [ ]:
pivoted_2[:100].to_csv("ForestFire/communities_backbone/networks_nmi_whatsapp_samples_normalized.csv")